In [1]:
from pathlib import Path
import numpy as np
import torch

TRAIN_FILEPATH = "/Users/33783/Desktop/capgemini/hackathon-mines-invent-2024/DATA/TRAIN"
TEST_FILEPATH = "/Users/33783/Desktop/capgemini/hackathon-mines-invent-2024/DATA/TEST"

BATCH_SIZE = 8

In [2]:
from baseline.dataset import BaselineDataset, BaselineDatatest
from baseline.collate import pad_collate

data_train_folder = Path(TRAIN_FILEPATH)
data_test_folder = Path(TEST_FILEPATH)

# Set seeds for PyTorch and numpy
torch.manual_seed(1234)
np.random.seed(1234)

dt_train = BaselineDataset(Path(data_train_folder))

# Split dataset: 80% train, 20% test
train_size = int(0.8 * len(dt_train))
val_size = len(dt_train) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dt_train, [train_size, val_size])

# Create DataLoaders
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=BATCH_SIZE, collate_fn=pad_collate, shuffle=True
    )
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=BATCH_SIZE, collate_fn=pad_collate, shuffle=False
    )

x_train, y_train = train_loader.__iter__().__next__()

for k, v in x_train.items():
    print(k,v.shape)
print("y", y_train.shape)

Reading patch metadata ...
Done.
Dataset ready.
S2 torch.Size([4, 61, 10, 128, 128])
date torch.Size([4, 61, 5])
TILE torch.Size([4, 6])
N_Parcel torch.Size([4, 1])
Parcel_Cover torch.Size([4, 1])
y torch.Size([4, 128, 128])


In [3]:
from sklearn.metrics import jaccard_score
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

import torch.nn.functional as F

from unet.model import UNET

MODEL_PATH = Path("models") / 'unet.pth'
LOAD_MODEL = False
IMG_HEIGHT = 128  
IMG_WIDTH = 128  
LEARNING_RATE = 0.0005
EPOCHS = 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
def preprocess_batch(input_batch):
    """
    Here we want to use the collapse the temporal dimension of the input
    batch by keeping only the first image of the month of august

    input_batch: dataloader X dict batch
    """

    L= []
    for i in range(input_batch["date"].shape[0]):
        L.append(torch.where(input_batch["date"][i,:,1] == 8)[0][0])
    indices_T_picked = torch.tensor(L)
    # Expand indices by creating a (B, 1, C, H, W) gather mask
    expanded_indices = indices_T_picked.view(-1, 1, 1, 1, 1).expand(-1, 1,
                                                                input_batch["S2"].size(2), input_batch["S2"].size(3),
                                                                input_batch["S2"].size(4))
    
    # Gather the values along the second dimension based on the indices
    collapsed_input_batch = torch.gather(input_batch["S2"], dim=1, index=expanded_indices).squeeze(1)

    return collapsed_input_batch

In [8]:
epoch = 0 # epoch is initially assigned to 0. If LOAD_MODEL is true then
              # epoch is set to the last value + 1. 
loss_values = [] # Defining a list to store loss values after every epoch

print('Data Loaded Successfully!')

# Defining the model, optimizer and loss function
model = UNET(in_channels=10, classes=20).to(device).train()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_function = nn.CrossEntropyLoss() 

# Loading a previous stored model from MODEL_PATH variable
if LOAD_MODEL == True:
    checkpoint = torch.load(MODEL_PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optim_state_dict'])
    epoch = checkpoint['epoch'] + 1
    loss_values = checkpoint['loss_values']
    print("Model successfully loaded!")

#Training the model for every epoch.
for e in range(epoch, EPOCHS):
    print(f'Epoch: {e}')
    running_loss = 0.0
    for i, (inputs, targets) in tqdm(enumerate(train_loader), total=len(train_loader)):
        inputs = preprocess_batch(inputs)
        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs)

        loss = loss_function(outputs, targets.long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss +=  loss.item()
        
    epoch_loss = running_loss / len(train_loader) / BATCH_SIZE
    loss_values.append(epoch_loss) 
    torch.save({
        'model_state_dict': model.state_dict(),
        'optim_state_dict': optimizer.state_dict(),
        'epoch': e,
        'loss_values': loss_values
    }, MODEL_PATH)
    print(f"loss: {epoch_loss}")
    print("Epoch completed and model successfully saved!")


Data Loaded Successfully!
Epoch: 0


  0%|          | 1/392 [00:00<04:46,  1.37it/s]

3.1245315074920654


  1%|          | 2/392 [00:01<04:15,  1.52it/s]

3.038583755493164


...

100%|█████████▉| 391/392 [02:16<00:00,  3.17it/s]

1.655423879623413


100%|██████████| 392/392 [02:17<00:00,  2.86it/s]

1.580357551574707


loss: 0.46734428071245854
Epoch completed and model successfully saved!


In [9]:
from sklearn.metrics import jaccard_score

def print_iou_per_class(
    targets: torch.Tensor,
    preds: torch.Tensor,
    nb_classes: int,
) -> None:
    """
    Compute IoU between predictions and targets, for each class.

    Args:
        targets (torch.Tensor): Ground truth of shape (B, H, W).
        preds (torch.Tensor): Model predictions of shape (B, nb_classes, H, W).
        nb_classes (int): Number of classes in the segmentation task.
    """

    # Compute IoU for each class
    # Note: I use this for loop to iterate also on classes not in the demo batch

    iou_per_class = []
    for class_id in range(nb_classes):
        iou = jaccard_score(
            targets == class_id,
            preds == class_id,
            average="binary",
            zero_division=0,
        )
        iou_per_class.append(iou)

    for class_id, iou in enumerate(iou_per_class):
        print(
            "class {} - IoU: {:.4f} - targets: {} - preds: {}".format(
                class_id, iou, (targets == class_id).sum(), (preds == class_id).sum()
            )
        )

def print_mean_iou(targets: torch.Tensor, preds: torch.Tensor) -> None:
    """
    Compute mean IoU between predictions and targets.

    Args:
        targets (torch.Tensor): Ground truth of shape (B, H, W).
        preds (torch.Tensor): Model predictions of shape (B, nb_classes, H, W).
    """

    mean_iou = jaccard_score(targets, preds, average="macro")
    print(f"meanIOU (over existing classes in targets): {mean_iou:.4f}")

In [12]:
score = 0.
for i, (inputs, targets) in tqdm(enumerate(val_loader)):
    inputs = preprocess_batch(inputs)
    inputs, targets = inputs.to(device), targets.to(device)
    outputs = model(inputs)
    preds = torch.argmax(outputs, dim=1) # Get the predicted class per pixel (B, H, W)

    preds = preds.cpu().numpy().flatten()
    targets = targets.cpu().numpy().flatten()
    score +=jaccard_score(targets, preds, average="macro")

print(f"meanIOU (over existing classes in targets): {score / len(val_loader)}")

1it [00:02,  2.14s/it]

0


2it [00:02,  1.21s/it]

1


3it [00:03,  1.04it/s]

...

97it [00:36,  1.91it/s]

96


98it [00:36,  2.67it/s]

97


ValueError: Invalid format specifier '.4f / 4' for object of type 'float'